# Завдання

*Завдання 1: Розбийте MNIST на тренувальну та навчальну вибірки (60k + 10k). Натренуйте Random Forest classifier ти виміряйте час тренування. Обчисліть точність на тестовій вибірці.*

In [1]:
import random
import time

from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
random.seed(42)

In [3]:
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 / 7, random_state=42)

In [4]:
print(f'{X_train.shape} {y_train.shape}')
print(f'{X_test.shape} {y_test.shape}')

(60000, 784) (60000,)
(10000, 784) (10000,)


In [5]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=100)

In [6]:
start_time_rfc = time.time()
rfc.fit(X_train, y_train)
end_time_rfc_pca = time.time()

In [7]:
train_time_rfc = end_time_rfc_pca - start_time_rfc
print(f'{round(train_time_rfc, 2)} seconds')

5.09 seconds


In [8]:
y_pred_rfc = rfc.predict(X_test)
print(classification_report(y_test, y_pred_rfc))
print(confusion_matrix(y_test, y_pred_rfc))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       983
           1       0.99      0.99      0.99      1152
           2       0.94      0.97      0.96       967
           3       0.96      0.95      0.95      1034
           4       0.96      0.97      0.97       906
           5       0.98      0.96      0.97       937
           6       0.98      0.99      0.98       961
           7       0.97      0.97      0.97      1055
           8       0.96      0.95      0.96       969
           9       0.96      0.94      0.95      1036

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

[[ 968    0    5    1    1    0    3    1    4    0]
 [   0 1136    5    6    1    0    0    1    2    1]
 [   4    2  941    1    4    0    4    4    7    0]
 [   1    0   19  980    0    6    1   12    8    7]
 [   2    0    1    0  880   

*Завдання 2: Застосуйте PCA з поясненою дисперсією 95%. Повторіть тренування. Порівняйте час тренування та точніть з попередньою вправою.*

In [9]:
pca = PCA(random_state=42, n_components=0.95)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [10]:
rfc_pca = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=100)

In [11]:
start_time_rfc_pca = time.time()
rfc_pca.fit(X_train_pca, y_train)
end_time_rfc_pca = time.time()

In [12]:
train_time_rfc_pca = end_time_rfc_pca - start_time_rfc_pca
print(f'{round(train_time_rfc_pca, 2)} seconds')

13.29 seconds


In [24]:
print(max(estimator.tree_.node_count for estimator in rfc.estimators_))
print(max(estimator.tree_.node_count for estimator in rfc_pca.estimators_))

10487
17043


In [14]:
y_pred_rfc_pca = rfc_pca.predict(X_test_pca)
print(classification_report(y_test, y_pred_rfc_pca))
print(confusion_matrix(y_test, y_pred_rfc_pca))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       983
           1       0.97      0.98      0.98      1152
           2       0.94      0.96      0.95       967
           3       0.92      0.94      0.93      1034
           4       0.93      0.95      0.94       906
           5       0.95      0.94      0.94       937
           6       0.96      0.97      0.97       961
           7       0.96      0.96      0.96      1055
           8       0.94      0.90      0.92       969
           9       0.95      0.92      0.93      1036

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

[[ 959    0    4    3    0    1    9    1    4    2]
 [   0 1129    8    5    0    0    1    2    4    3]
 [   2    4  926    3    5    1    4    7   14    1]
 [   0    2   15  968    0   16    2   13   12    6]
 [   4    3    0    0  862   

Тренування RandomForestClassifier (RFC) зайняло 5.09 секунд, а тренування на сеті зі зменшеною розмірністю зайняло більше - 13.29 секунд. Це можна пояснити більшою к-стю створених nodes у RFC з PCA. Якщо зменшити к-сть дерев `n_estimators` для RFC з PCA до, наприклад, 50, то звісно час тренування зменшиться, але і точність разом з тим, хоча і не на критичне значення. Щодо точності, то і в простому RFC, і RFC з PCA вона відмінна, хоча у першому найкраща серед усіх моделей у цьому завданні, але це очікувано, через зменшення розмірності сету даних.

*Завдання 3: Повторіть ці ж кроки з логістичною регресією. Зробіть висновки.*

In [15]:
lr = LogisticRegression(random_state=42, n_jobs=-1, max_iter=100)

In [16]:
start_time_lr = time.time()
lr.fit(X_train, y_train)
end_time_lr = time.time()

In [17]:
train_time_lr = end_time_lr - start_time_lr
print(f'{round(train_time_lr, 2)} seconds')

24.28 seconds


In [18]:
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       983
           1       0.95      0.97      0.96      1152
           2       0.90      0.89      0.90       967
           3       0.88      0.91      0.89      1034
           4       0.92      0.92      0.92       906
           5       0.90      0.85      0.87       937
           6       0.93      0.95      0.94       961
           7       0.94      0.94      0.94      1055
           8       0.88      0.87      0.88       969
           9       0.91      0.91      0.91      1036

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

[[ 941    0    5    1    3   10   11    4    7    1]
 [   0 1123    4    6    1    5    0    2    9    2]
 [   3   15  865   20   10    4   17    8   22    3]
 [   2    5   27  937    1   25    2   10   13   12]
 [   3    1    4    6  837   

In [19]:
lr_pca = LogisticRegression(random_state=42, n_jobs=-1, max_iter=100)

In [20]:
start_time_lr_pca = time.time()
lr_pca.fit(X_train_pca, y_train)
end_time_lr_pca = time.time()

In [21]:
train_time_lr_pca = end_time_lr_pca - start_time_lr_pca
print(f'{round(train_time_lr_pca, 2)} seconds')

8.25 seconds


In [22]:
y_pred_lr_pca = lr_pca.predict(X_test_pca)
print(classification_report(y_test, y_pred_lr_pca))
print(confusion_matrix(y_test, y_pred_lr_pca))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96       983
           1       0.92      0.97      0.95      1152
           2       0.91      0.90      0.90       967
           3       0.89      0.88      0.89      1034
           4       0.90      0.92      0.91       906
           5       0.89      0.87      0.88       937
           6       0.92      0.95      0.94       961
           7       0.92      0.94      0.93      1055
           8       0.91      0.83      0.87       969
           9       0.90      0.90      0.90      1036

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

[[ 944    0    5    0    4   10    9    5    4    2]
 [   0 1119    4    6    0    6    1    5   10    1]
 [   8   20  867   11   11    4   19   11   14    2]
 [   2   12   27  915    1   30    9   14   13   11]
 [   5    3    4    3  834   

Тренування LogisticRegression без застосування PCA зайняло найбільше часу - 24.28 секунд, але з PCA - 8.25, зменшився у 3 рази. При цьому точність майже не змінилася, порівнюючи між моделями логістичної регресії.

PCA, у висновку, значно зменшив час тренування логістичної регресії, проте збільшив для Random Forest Classifier, хоча тренування RFC на оригінальному сеті даних займає найменше часу. У точності ж будь-яка логістична регресія програє RFC, хоча і втрати не є великими.